In [ ]:
import tensorrt as trt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization,Rescaling
from tensorflow.keras.optimizers import Adam
from efficientnet.tfkeras import EfficientNetB3
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

import matplotlib.pyplot as plt
import os

#use if using gpu
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*4)])
  except RuntimeError as e:
    print(e)

# Set the path to your dataset
train_data_dir = os.path.join(os.getcwd(),'dataset/train')
test_data_dir = os.path.join(os.getcwd(),'dataset/test')
# Define parameters
img_width, img_height = 300, 300  # EfficientNet input size
input_shape = (img_width, img_height, 3)
epochs = 30
batch_size = 32

train_generator=tf.keras.utils.image_dataset_from_directory(train_data_dir,validation_split=0.2,
                                                            seed=123,subset="training",
                                                            image_size=(img_width, img_height),
                                                            batch_size=batch_size,
                                                            label_mode='categorical'
                                                            )

val_generator=tf.keras.utils.image_dataset_from_directory(train_data_dir,
                                                          validation_split=0.2,
                                                          seed=123,subset="validation",
                                                          image_size=(img_width, img_height),
                                                          batch_size=batch_size,
                                                          label_mode='categorical')

# Use EfficientNetB3 as a base model
base_model = EfficientNetB3(weights='imagenet', include_top=False)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model on top of the pre-trained base model
model = Sequential()
model.add(Rescaling(1./255,input_shape=input_shape))
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # Assuming you have 7 classes

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Learning rate schedule
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=[reduce_lr, early_stop]
    )

# Save the model
model.save('car_brand_classifier_advanced_model.h5')

# Plot training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import os
from tensorflow.keras.preprocessing import image
model=load_model('car_brand_classifier_advanced_model.h5')
imsize=(300,300)
labels=['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce', 'Swift', 'Tata Safari', 'Toyota Innova']

In [ ]:
import numpy as np

predicted_label=[]
true_label=[]
for i in sorted(os.listdir('dataset/test')):
    for j in os.listdir(os.path.join('dataset/test',i)):
        filename=os.path.join('dataset/test/',i,j)
        img=image.load_img(filename, target_size=imsize)
        resized_img=image.img_to_array(img)
        final=np.expand_dims(resized_img,axis=0)
        predictions=model.predict(final)
        true_label.append(sorted(os.listdir('dataset/test')).index(i))
        predicted_label.append(list(predictions[0]).index(max(predictions[0])))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Create confusion matrix
cm = confusion_matrix(true_label, predicted_label)
cm_percentages = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
# Plot the confusion matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm_percentages, annot=True, fmt="d", cmap="Blues", xticklabels=labels,
            yticklabels=labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix for 7 Classes')
plt.show()